In [13]:
import pyspark
import json
from collections import defaultdict
from copy import deepcopy

In [14]:
conf = pyspark.SparkConf()
conf.setAppName("Twitter page rank")
conf.setMaster("local[*]")

In [15]:
sc = pyspark.SparkContext(conf = conf)
#sc

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Twitter page rank, master=local[*]) created by __init__ at C:\Users\hdelc\AppData\Local\Temp\ipykernel_12724\2208299540.py:1 

In [ ]:
twitterLines = sc.textFile("twitter_closed.jsonl")
#twitterLines.collect()
lineas = twitterLines.collect()
#len(lineas),type(lineas[0])
#lineas[0]
twitterLines


twitter_closed.jsonl MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [ ]:
seguidos = twitterLines.map(lambda linea: json.loads(linea))
####seguidos = twitterLines.map(json.loads)
#seguidos.collect()

In [ ]:
links = seguidos.keyBy(lambda diccionario: diccionario["user_id"]).mapValues(lambda diccionario: diccionario["following_id"])
#links.collect()[1]



In [ ]:
N = 1000#links.count()
d = 0.85

In [ ]:
ranks = links\
    .keyBy(lambda usuario: usuario[0])\
    .map(lambda usuario: (usuario[0], 1 / N))

In [ ]:
iterations = 10

In [ ]:
def calcula_contribuciones(registro):
    # Calcula las contribuciones de un `usuario` hacia las personas a las que sigue

    
    resultados = list()
    (usuario, (seguidos, page_rank_usuario)) = registro

    for seguido in seguidos:
        resultados.append((seguido, page_rank_usuario / len(seguidos)))
    resultados.append((usuario, 0))
    
    return resultados

In [ ]:
for i in range(iterations):
    
    contribuciones = links.join(ranks)\
        .flatMap(calcula_contribuciones)
    
    ranks = contribucioness\
        .reduceByKey(lambda a, b: a + b)\
        .mapValues(lambda v: (1 - d) / N + d * v)

In [ ]:
results = dict(ranks.collect())

In [ ]:
results[2582486138]

In [ ]:
results[1306760288243462147]

In [ ]:
sc.stop()